## **5장 컴퓨터 비전을 위한 딥러닝**

**이 장에서는 컨브넷(convnet)이라고도 불리는 합성곱 신경망(convolutional neural network)을 배운다. 이 딥러닝 모델은 거의 대부분의 컴퓨터 비전(computer vision) 애플리케이션에 사용된다.**

**거대 IT회사가 아니라면 대부분 작은 데이터셋을 다루므로 여기에서도 작은 훈련 데이터셋을 사용한 이미지 분류 문제에 컨브넷을 적용하는 법을 배운다.**

## **5.1 합성곱 신경망 소개**

**컨브넷 정의와 컨브넷이 컴퓨터 비전 관련 작업에 잘 맞는 이유에 대해 이론적 배경을 알아보기 전에 간단한 컨브넷 예제를 돌려본다.**

**기본적인 컨브넷이더라도 2장에서 다룬 완전 연결된 모델의 성능을 훨씬 앞지를 것이다.**

In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from IPython.display import display
import PIL
from keras import backend as K

Using TensorFlow backend.


In [ ]:
K.tensorflow_backend._get_available_gpus()

In [ ]:
K.tensorflow_backend._get_available_gpus()

In [2]:
import tensorflow as tf

In [3]:
from keras import layers
from keras import models

In [4]:
# 기본적인 컨브넷의 모습. Conv2D와 MaxPooling2D 층을 쌓아 올렸다.
model = models.Sequential()

In [5]:
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))

In [6]:
# 지금까지의 컨브넷 구조를 출력
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


**컨브넷이 (image_height, image_width, image_channels) 크기의 입력 텐서를 사용한다는 점이 중요하다.(배치 차원은 포함하지 않는다.)**

> **이 때문에 첫 번째 층의 매개변수로 input_shape=(28,28,1)을 전달**

**Conv2D와 MaxPooling2D 층의 출력은 (height, width, channels) 크기의 3D 텐서이다. 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있다. 채널의 수는 Conv2D 층에 전달된 첫 번째 매개변수에 의해 조절된다.(32개 또는 64개)**

**마지막 층의 ((3,3,64) 크기인) 출력 텐서를 완전 연결 네트워크에 주입한다.**

> **이 네트워크는 이미 익숙하게 보았던 Dense 층을 쌓은 분류기이다. 이 분류기는 1D 벡터를 처리하는데, 이전 층의 출력이 3D 텐서이다. 그래서 먼저 3D 출력을 1D 텐서로 펼쳐야한다.**

In [7]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10,activation='softmax'))

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

**10개의 클래스를 분류하기 위해 마지막 층의 출력 크기를 10으로 하고 소프트맥스 활성화 함수를 사용한다.**

**여기에서 볼 수 있듯이 (3,3,64) 출력이 (576,) 크기의 벡터로 펼쳐진 후 Dense 층으로 주입되었다.**

In [9]:
from keras.datasets import mnist
from keras.utils import to_categorical

In [10]:
# 데이터 불러오기
(train_images, train_labels),(test_images, test_labels) = mnist.load_data()

# 훈련 이미지를 3차원 이미지로 reshape
train_images = train_images.reshape((60000,28,28,1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000,28,28,1))
test_images = test_images.astype('float32') / 255

# labels 데이터 one_hot_encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# model compile
model.compile(optimizer='rmsprop',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

model.fit(train_images,train_labels,epochs=20,batch_size=64)

Epoch 1/20
60000/60000 [==============================] - 8s 138us/step - loss: 0.1760 - acc: 0.9453
Epoch 2/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0483 - acc: 0.9855
Epoch 3/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0329 - acc: 0.9896
Epoch 4/20
60000/60000 [==============================] - 3s 52us/step - loss: 0.0256 - acc: 0.9920
Epoch 5/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0193 - acc: 0.9940: 0s - loss: 0
Epoch 6/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0162 - acc: 0.9951
Epoch 7/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0124 - acc: 0.9964
Epoch 8/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0103 - acc: 0.9969
Epoch 9/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.0091 - acc: 0.9972
Epoch 10/20
60000/60000 [==============================] - 3s 53us/step - loss: 0.007

In [ ]:
test_loss, test_acc = model.evaluate(test_images,test_labels)
test_acc

**2장의 완전 연결 네트워크는 97.8%의 테스트 정확도를 얻은 반면에 기본적인 컨브넷은 99.2%의 테스트 정확도를 얻었다. 에려율이 (상대적으로) 64%나 줄었다.**

### **5.1.1 합성곱 연산**

**완전 연결 층과 합성곱 층 사이의 근본적인 차이는 다음과 같다. Dense층은 입력 특성 공간에 있는 전역 패턴(예를 들어 MNIST 숫자 이미지에서는 모든 픽셀에 걸친 패턴)을 학습하지만 합성곱 층은 지역 패턴을 학습한다.**

> **이미지일 경우 작은 2D 윈도우로 입력에서 패턴을 찾는다.(앞의 예에서 이 윈도우는 모두 3X3 크기였다.)**

![test](./img/합성곱.png)

**이 핵심 특징은 컨브넷에 두 가지 흥미로운 성질을 제공한다.**

> **학습된 패턴은 평행 이동 불변성을 가진다. 컨브넷이 이미지 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳(예를 들어 왼쪽 위 모서리)에서도 이 패턴을 인식할 수 있다. 완전 연결 네트워크는 새로운 위치에 나타난 것을 새로운 패턴으로 학습해야 한다. 이런 성질은 컨브넷이 이미지를 효율적으로 처리하게 만들어 준다.(근본적으로 우리가 보는 세상은 평행 이동으로 인해 다르게 인식 되지 않습니다.)적은 수의 훈련 샘플을 사용해서 일반화 능력을가진 표현을 학습할 수 있다.**

> **컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있다. 첫 번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습한다. 두 번째 합성곱 층은 첫 번째 층의 특성으로 구성된 더 큰 패턴을 학습하는 식이다. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적으로 학습할 수 있다.(근본적으로 우리가 보는 세상은 공간적 계층 구조를 가진다.)**

**합성곱 연산은 특성 맵(feature map)이라고 부르는 3D 텐서에 적용된다.**

> **이 텐서는 2개의 공간축(높이와 너비)과 깊이 축(채널 축이라고도 한다)으로 구성된다. RGB 이미지는 3개의 컬러 채널(빨간색, 녹색, 파란색)을 가지므로 깊이 축의 차원이 3이 된다.**

> **MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 1(회색 톤)입니다.**

**합성곱 연산은 입력 특성 맵에서 작은 패치(Petch)들을 추출하고 이런 모든 패치에 같은 변환을 적용하여 출력 특성 맵(output feature map)을 만든다.**

![test](./img/CNN2.png)

**출력 특성 맵도 높이와 너비를 가진 3D 텐서이다. 출력 텐서의 깊이는 층의 매개변수로 결정되기 떄문에 상황에 따라 다르다. 이렇게 되면 깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하지 않는다. 그 대신 일종의 필터를 의미한다.**

> **여기서 필터는 합성곱 층에서 사용하는 모델 파라미터를 의미한다. Conv2D의 첫 번째 매개변수(필터 또는 채널 수)가 출력 특성 맵의 깊이 차원을 결정한다.**

> **필터는 입력 데이터의 어떤 특성을 인코딩한다. 예를 들어 고수준으로 보면 하나의 필터가 입력에 얼굴이 있는지를 인코딩 할 수 있다.**

**MNIST 예제에서는 첫 번째 합성곱 층이 (28,28,1) 크기의 특성 맵을 입력으로 받아 (26,26,32)크기의 특성 맵을 출력한다. 즉 입력에 대해 32개의 필터를 적용한다.**

> **32개의 출력 채널은 각각 26X26 크기의 배열 값을 가진다. 이 값은 입력에 대한 필터의 응답 맵(response map)이다. 입력의 각 위치에서 필터 패턴에 대한 응답을 나타낸다.**

> **특성 맵이란 말이 의미하는 것은 다음과 같다. 깊이 축에 있는 각 차원은 하나의 특성(또는 필터)이고, 2D 텐서 output[;,;,n]은 입력에 대한 이 필터 응답을 나타내는 2D 공간상의 맵이다.**

![test](./img/mnist.jpg)

**합성곱은 핵심적인 2개의 파라미터로 정의된다.**

> **입력으로부터 뽑아낼 패치의 크기: 전형적으로 3x3 또는 5x5 크기를 사용한다. 이 예에서는 일반적으로 많이 사용하는 3x3 크기를 사용했다.**

> **특성 맵의 출력 깊이: 합성곱으로 계산할 필터의 수이다. 이 예에서는 깊이 32로 시작해서 깊이 64로 끝났다.**

**케라스의 Conv2D 층에서 이 파라미터는 Conv2D(output_depth,(window_height,window_width)) 처럼 첫 번째와 두 번째 매개변수로 전달된다.**

> **3D 입력 특성 맵 위를 3x3 또는 5x5 크기의 윈도우가 슬라이딩(sliding)하면서 모든 위치에서 3D 특성 패치((window_height,window_width,input_depth)크기)를 추출하는 방식으로 합성곱이 작동한다.**

> **이런 3D 패치는 (output_depth,) 크기의 1D 벡터로 변환된다. (합성곱 커널 Convoluation kernel)이라고 불리는 하나의 학습된 가중치 행렬과의 텐서 곱을 통해 변환된다.)**

> **변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵으로 재구성된다.**

> **출력 특성 맵의 공간상 위치는 입력 특성 맵의 같은 위치에 대응된다.(예를 들어 출력의 오른쪽 아래 모서리는 입력의 오른쪽 아래 부근에 해당하는 정보를 담고 있다.)**

> **3x3 윈도우를 사용하면 3D 패치 input[i-1:i+2,j-1,j+2,:]로부터 벡터 output[i,j,:]가 만들어진다.**

![test](./img/합성곱1.png)

![test](./img/합성곱2.jpg)

![test](./img/합성곱3.png)

**출력 높이와 너비는 입력의 높이, 너비와 다를 수 있다. 여기에는 두 가지 이유가 있다.**

> **경계 문제, 입력 특성 맵에 패딩을 추가하여 대응할 수 있다.**

> **잠시 후에 설명할 스트라이드(stride)의 사용 여부에 따라 다르다.**

### **경계 문제와 패딩 이해하기**

![test](./img/패딩계산.png)

**5x5 크기의 특성 맵을 생각했을때(총 25개의 타일이 있다고 가정). 3x3 크기인 윈도우의 중앙을 맞출 수 있는 타일은 3x3 격자를 형성하는 9개뿐인다. 따라서 출력 특성 맵은 3x3 크기가 된다.**

> **크기가 조금 줄어들었다. 여기에서는 높이와 너비 차원에 따라 2개의 타일이 줄어들었다.**

**입력과 동일한 높이와 너비를 가진 출력 특성 맵을 얻고 싶다면 패딩을 사용할 수 있다. 패딩은 입력 특성 맵의 가장자리에 적절한 개수의 행과 열을 추가한다. 그래서 모든 입력 타일에 합성곱 윈도우의 중앙을 위치시킬 수 있다.**

**Conv2D 층에서 패딩은 padding 매개변수로 설정할 수 있다. 2개의 값이 가능하다.**

> **'valid'는 패딩을 사용하지 않는다는 뜻이다.(윈도우를 놓을 수 있는 위치만 사용한다.)**

> **'same'은 입력과 동일한 높이와 너비를 가진 출력을 만들기 위해 패딩한다.**

> **padding 매개변수의 기본값은 'valid'이다.**

### **합성곱 스트라이드 이해하기**

**출력 크기에 영향을 미치는 다른 요소는 스트라이드이다. 지금까지 합성곱에 대한 설명은 합성곱 윈도우의 중앙 타일이 연속적으로 지나간다고 가정한 것이다.**

> **두 번의 연속적인 윈도우 사이의 거리가 스트라이드라고 불리는 합성곱의 파라미터이다.**

**스트라이드의 기본값은 1이고, 스트라이드가 1보다 큰 스트라이드 합성곱도 가능하다.**

> **예를 들어 스트라이드 2를 사용했다는 것은 특성 맵의 너비와 높이가 2의 배수로 다운샘플링되었다는 뜻이다.(경계 문제가 있다면 더 줄어든다.)**

**스트라이드 합성곱은 실전에서 드물게 사용된다. 하지만 어떤 모델에서는 유용하게 사용될 수 있으므로 잘 알아 둘 필요가 있다.**

![test](./img/스트라이드.gif)

**특성 맵을 다운샘플링하기 위해서 스트라이드 대신에 첫 번째 컨브넷 예제에 사용된 최대 출링(max pooling) 연산을 사용하는 경우가 많다.**

### **5.1.2 최대 풀링 연산**

**앞선 컨브넷 예제에서 특성 맵의 크기가 MaxPooling2D 층마다 절반으로 줄어들었다.**

> **예를 들어 첫 번째 MaxPooling2D 층 이전에 특성 맵의 크기는 26x26이었는데 최대 풀링 연산으로 13x13으로 줄어들었다.**

**스트라이드 합성곱과 매우 비슷하게 강제적으로 특성 맵을 다운샘플링하는 것이 최대 풀링의 역할이다.**

**최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최대값을 출력한다.**

> **합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환(합성곱 커널)을 적용하는 대신 하드코딩된 최댓값 추출 연산을 사용한다.**

**합성곱과 가장 큰 차이점은 최대 풀링은 보통 2x2 윈도우와 스트라이드 2를 사용하여 특성 맵을 절반 크기로 다운샘플링을 한다는 것이다.**

> **이에 반해 합성곱은 전형적으로 3x3 윈도우와 스프라이드 1을 사용한다.**

In [ ]:
# 모델 생성
model_no_max_pool = models.Sequential()

In [ ]:
# 풀링 계층없이 합성곱으로만 이루어진 모델
model_no_max_pool.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(28,28,1)))
model_no_max_pool.add(layers.Conv2D(64,(3,3), activation='relu'))
model_no_max_pool.add(layers.Conv2D(64,(3,3), activation='relu'))

In [ ]:
model_no_max_pool.summary()

**이 설정에는 두 가지 문제가 있다.**

> **특성의 공간적 계층 구조를 학습하는 데 도움이 되지 않는다. 세 번째 층의 3x3 윈도우는 초기 입력의 7x7 윈도우 영역에 대한 정보만 담고 있다. 컨브넷에 의해 학습된 고수준 패턴은 초기 입력에 관한 정보가 아주 적어 숫자 분류를 학습하기에 충분하지 않을 것이다. 마지막 합성곱 층의 특성이 전체 입력에 대한 정보를 가지고 있어야 한다.**

> **최종 특성 맵은 22x22x64 = 30,976개의 가중치를 가진다. 즉, 아주 많다. 이 컨브넷을 펼친 후 512크기의 Dense 층과 연결한다면 약 15.8백만 개의 가중치 파라미터가 생긴다. 작은 모델치고는 너무 많은 가중치고, 심각한 과대적합이 발생한다.**

**간단히 말해서 다운샘플링을 사용하는 이유는 처리할 특성 맵의 가중치 개수를 줄이기 위해서이다. 또 연속적인 합성곱 층이 (원본 입력에서 커버되는 영역 측면에서) 점점 커진 윈도우를 통해 바라보도록 만들어 필터의 공간적인 계층 구조를 구성한다.**

**최대 풀링이 다운샘플링을 할 수 있는 유일한 방법은 아니다. 이미 알고 있듯이 앞선 합성곱 층에서 스트라이드를 사용할 수 있다.**

**최대값을 취하는 최대 풀링 대신에 입력 패치의 채널별 평균값을 계산하여 변환하는 평균 풀링(average pooling)을 사용할 수도 있다. 하지만 최대 풀링이 다른 방법들보다 더 잘 작동하는 편이다. 그 이유는 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩하는 경향이 있기 때문이다.(그래서 특성의 지도(맵)이다.)**

**따라서 특성의 평균값보다 여러 특성 중 최대값을 사용하는 것이 유용하다.**

**가장 납득할 만한 서브샘플링(Subsampling)전략은 먼저 (스트라이드가 없는 합성곱으로) 조밀한 특성 맵을 만들고 그다음 작은 패치에 대해서 최대로 활성화된 특성을 고르는 것이다.**

> **이런 방법이 입력에 대해 (스트라이드 합성곱으로) 듬성듬성 윈도우를 슬라이딩하거나 입력 패치를 평균해서 특성 정보를 놓치거나 희석시키는 것보다 낫다.**

## **5.2 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기**

**이 절에서 문제를 해결하기 위해 기본적인 전략을 하나를 살펴본다. 보유한 소규모 데이터셋을 사용하여 처음부터 새로운 모델을 훈련하는 것이다.**

> **먼저 아무런 규제를 하지 않고 훈련하여 기준이 되는 기본 성능을 만든다. 이 방법의 주요 이슈는 과대적합이 될 것이다.**

> **그 다음 컴퓨터 비전에 과대적합을 줄이기 위한 강력한 방법인 데이터 증식(data augmentation)을 하여 네트워크의 성능을 82% 정확도로 향상 시킨다.**

> **사전 훈련된 네트워크로 특성을 추출하는 것과 사전 훈련된 네트워크를 세밀하게 튜닝하여 최종 모델은 92% 정확도를 얻는다.**

### **5.2.1 작은 데이터셋 문제에서 딥러닝의 타당성**

**딥러닝은 데이터가 풍부할 떄만 작동한다는 말을 이따금 듣는다. 부분적으로는 맞지만 딥러닝의 근본적인 특징은 훈련 데이터에서 특성 공학의 수작업 없이 흥미로운 특성을 찾을 수 있는 것이다. 이는 훈련 샘플이 많아야만 가능하다.**

> **특히 입력 샘플이 이미지처럼 매우 고차원적인 문제에서는 특히 그렇다.**

**하지만 많은 샘플이 의미하는 것은 상대적이다. 우선 훈련하려는 네트워크의 크기와 깊이가 상대적이다. 복잡한 문제를 푸는 컨브넷을 수십 개의 샘플만 사용해서 훈련하는 것은 불가능하다.**


**하지만 모델이 작고 규제가 잘 되어 있으며 간단한 작업리마녀 수백 개의 샘플로도 충분할 수 있다. 컨브넷은 지역적이고 평행 이동으로 변하지 않는 특성을 학습하기 때문에 지각에 관한 문제에서 매우 효율적으로 데이터를 사용한다.**

> **매우 작은 이미지 데이터셋에서 어떤 종류의 특성 공학을 사용하지 않고 컨브넷을 처음부터 훈련해도 납득할 만한 결과를 만들 수 있다.**

**거기에 더하여 딥러닝 모델은 태생적으로 매우 다목적이다. 말하자면 대규모 데이터셋에서 훈련시킨 이미지 분류 모델이나 스피치-투-텍스트(speech-to-text) 모델을 조금만 변경해서 완전히 다른 문제에 재사용할 수 있다.**

> **특히 컴퓨터 비전에서는 보통 사전 훈련된 모델들이 내려받을 수 있도록 많이 공개되어 있어서 매우 적은 데이터에서 강력한 비전 모델을 만드는 데 사용할 수 있다.**

**이 데이터셋은 2만 5,000개의 강아지와 고양이 이미지(클래스마다 1만 2,500개)를 담고 있고 (압축해서) 543MB 크기이다. 클래스마다 1,000개의 샘플로 이루어진 훈련 세트, 클래스마다 500개의 샘플로 이루어진 검증 세트, 클래스마다 500개의 샘플로 이루어진 테스트 세트이다.**

In [ ]:
import os, shutil

In [ ]:
# 원본 데이터셋을 압축 해제한 디렉터리 경로
original_dataset_dir = './dogs-vs-cats/train/train'

In [ ]:
# 소규모 데이터셋을 저장할 디렉터리
base_dir = './cats_and_dogs_small'
os.mkdir(base_dir)

In [ ]:
# 훈련, 검증, 테스트 분할을 위한 디렉터리
train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir,'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir,'test')
os.mkdir(test_dir)

In [ ]:
# 훈련용 고양이 사진 디렉터리
train_cats_dir = os.path.join(train_dir,'cats')
os.mkdir(train_cats_dir)

# 훈련용 강아지 사진 디렉터리
train_dogs_dir = os.path.join(train_dir,'dogs')
os.mkdir(train_dogs_dir)

# 검증용 고양이 사진 디렉터리
validation_cats_dir = os.path.join(validation_dir,'cats')
os.mkdir(validation_cats_dir)

# 검증용 강아지 사진 디렉터리
validation_dogs_dir = os.path.join(validation_dir,'dogs')
os.mkdir(validation_dogs_dir)

# 테스트용 고양이 사진 디렉터리
test_cats_dir = os.path.join(test_dir,'cats')
os.mkdir(test_cats_dir)

# 테스트용 강아지 사진 디렉터리
test_dogs_dir = os.path.join(test_dir,'dogs')
os.mkdir(test_dogs_dir)

# 처음 1,000개의 고양이 이미지를 train_cats_dir에 복사한다.
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(train_cats_dir,fname)
    shutil.copyfile(src,dst)

# 다음 500개의 고양이 이미지를 validation_cats_dir에 복사한다.
fnames = ['cat.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(validation_cats_dir,fname)
    shutil.copyfile(src,dst)

# 다음 500개의 고양이 이미지를 test_cat_dir에 복사
fnames = ['cat.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(test_cats_dir,fname)
    shutil.copyfile(src,dst)
    
# 처음 1,000개의 강아지 이미지를 train_cats_dir에 복사한다.
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(train_dogs_dir,fname)
    shutil.copyfile(src,dst)

# 다음 500개의 강아지 이미지를 validation_cats_dir에 복사한다.
fnames = ['dog.{}.jpg'.format(i) for i in range(1000,1500)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(validation_dogs_dir,fname)
    shutil.copyfile(src,dst)

# 다음 500개의 강아지 이미지를 test_cat_dir에 복사
fnames = ['dog.{}.jpg'.format(i) for i in range(1500,2000)]
for fname in fnames:
    src = os.path.join(original_dataset_dir,fname)
    dst = os.path.join(test_dogs_dir,fname)
    shutil.copyfile(src,dst)


In [ ]:
# 복사가 잘 되었는지 확인하기 위해 각 분할에 들어 있는 사진의 개수를 카운트

print('훈련용 고양이 이미지 전체 개수:',len(os.listdir(train_cats_dir)))
print('훈련용 강아지 이미지 전체 개수:',len(os.listdir(train_dogs_dir)))
print('검증용 고양이 이미지 전체 개수:',len(os.listdir(validation_cats_dir)))
print('검증용 강아지 이미지 전체 개수:',len(os.listdir(validation_dogs_dir)))
print('테스트용 강아지 이미지 전체 개수:',len(os.listdir(test_cats_dir)))
print('테스트용 고양이 이미지 전체 개수:',len(os.listdir(test_dogs_dir)))

**이제 2,000개의 훈련 이미지 , 1,000개의 검증 이미지, 1,000개의 테스트 이미지가 준비되었다. 분할된 각 데이터는 클래스마다 동일한 개수의 샘플을 포함한다. 균형 잡힌 이진 분류 문제이므로 정확도를 사용하여 성공을 측정**

### **5.2.3 네트워크 구성하기**

**이전보다 이미지가 크고 복잡한 문제이기 때문에 네트워크를 좀 더 크게 만든다. Conv2D + MaxPooling2D 단계를 하나 더 추가한다. 이렇게 하면 네트워크의 용량을 늘리고 Flatten층의 크기가 너무 커지지 않도록 특성 맵의 크기를 줄일 수 있다.**

> **150 X 150 크기(임의로 선택한 것)의 입력으로 시작해서 Flatten 층 이전에 7 X 7 크기의 특성 맵으로 줄어든다.**

> **특성 맵의 깊이는 네트워크에서 점진적으로 증가하지만 (32에서 128까지), 특성 맵의 크기는 감소한다. (150 X 150에서 7 X 7까지). 이는 거의 모든 컨브넷에서 볼 수 있는 전형적인 패턴이다.**

In [ ]:
from keras import layers
from keras import models

# 모델 생성
model = models.Sequential()
model.add(layers.Conv2D(32,(3,3), activation='relu',
                       input_shape = (150,150,3)))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(128,(3,3), activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

**컴파일 단계에서 이전과 같이 RMSprop 옵티마이저를 선택한다. 네트워크의 마지막 하나의 시그모이드 유닛이기 때문에 이진 크로스엔트로피를 손실로 사용한다.**

In [ ]:
from keras import optimizers

model.compile(loss = 'binary_crossentropy',
             optimizer = optimizers.RMSprop(lr=1e-4),
             metrics = ['acc'])

### **5.2.4 데이터 전처리**

**데이터는 네트워크에 주입되기 전에 부동 소수 타입의 텐서로 적절하게 전처리되어 있어야 한다. 지금은 데이터가 JPEG 파일로 되어 있으므로 네트워크에 주입하려면 대략 다음 과정을 따른다.**

> **1. 사진 파일을 읽는다.**

> **2. JPEG 콘텐츠를 RGB 픽셀 값으로 디코딩한다.**

> **3. 그다음 부동 소수 타입의 텐서로 변환한다.**

> **4. 픽셀 값(0에서 255사이)의 스케일을 [0,1] 사이로 조정한다.(신경망은 작은 입력 값을 선호한다.)**

**케라스에서는 이런 단계를 자동으로 처리하는 유틸리티가 있다. 또한 케라스에는 keras.preprocessing.image에 이미지 처리를 위한 헬퍼 도구들도 있다.**

**특히 ImageDataGenerator 클래스는 디스크에 있는 이미지 파일을 전처리된 배치 텐서로 자동으로 바꾸어 주는 파이썬 제너레이터를 만들어 준다.**

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# 모든 이미지를 1/255로 스케일을 조정한다.
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir, # 타깃 디렉토리
    target_size = (150,150), # 모든 이미지를 (150,150)으로 바꾼다.
    batch_size = 20, 
    class_mode = 'binary' # binary_crossentropy 손실을 사용하기 때문에 이진 레이블 필요
)
validation_generator = test_datagen.flow_from_directory(
    validation_dir, # 타깃 디렉토리
    target_size = (150,150), # 모든 이미지를 (150,150)으로 바꾼다.
    batch_size = 20, 
    class_mode = 'binary' # binary_crossentropy 손실을 사용하기 때문에 이진 레이블 필요
)

In [ ]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:',data_batch.shape)
    print('배치 레이블 크기:',labels_batch.shape)
    break

**이 제너레이터의 출력은 150 X 150 RGB 이미지의 배치 (20, 150, 150, 3) 크기와 이진 레이블의 배치 (20,)이다. 각 배치에는 20개의 샘플(배치 크기)이 있다. 제너레이터는 이 배치를 무한정 만들어 내므로 타깃 폴더에 있는 이미지를 끝없이 반복한다. 따라서 반복 루프 안 어디에선가 break문을 사용해야 한다.**

**fit_generator 메서드는 fit 메서드와 동일하되 데이터 제너레이터를 사용할 수 있다.**

> **이 메서드는 첫 번째 매개변수로 입력과 타깃의 배치를 끝없이 반환하는 파이썬 제너레이터를 기대한다.**

> **데이터가 끝없이 생성되기 때문에 케라스 모델에 하나의 에포크를 정의하기 위해 제너레이터로부터 얼마나 많은 샘플을 뽑을 것인지 알려주어야 한다. steps_per_epoch 매개변수에서 이를 설정**

> **제너레이터로부터 steps_per_epoch개의 배치만큼 뽑은 후, 즉 steps_per_epoch 횟수만큼 경사 하강법 단계를 실행한 후에 훈련 프로세스는 다음 에포크로 넘어간다.**

> **여기서는 20개의 샘플이 하나의 배치이므로 2,000개의 샘플을 모두 처리할 때까지 100개의 배치를 뽑을 것이다.**

**fit_generator를 사용할 때 fit 메서드와 마찬가지로 validation_data 매개변수를 전달할 수 있다. 이 매개변수에는 데이터 제너레이터도 가능하지만 너마일 배열의 튜플도 가능하다.**

> **validation_data로 제너레이터를 전달하면 검증 데이터의 배치를 끝없이 반환한다.**

> **따라서 검증 데이터 제너레이터에서 얼마나 많은 배치를 추출하여 평가할지 validation_steps 매개변수에 지정해야 한다.**

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = 100,
    epochs = 30,
    validation_data = validation_generator,
    validation_steps = 50
)